In [1]:
import torch
import transformers

In [2]:
print("Pytorch:      ", torch.__version__)
print("Transformers: ", transformers.__version__)
print("GPU available:", torch.cuda.is_available())
print("GPU name:     ", torch.cuda.get_device_name(0))

Pytorch:       1.3.0
Transformers:  2.1.1
GPU available: True
GPU name:      GeForce GTX 1080 Ti


# Model: Distilled GPT-2 (6 layer, 82M params)

In [ ]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilgpt2')
model     = transformers.DistilBertModel.from_pretrained('distilgpt2')
modelLM   = transformers.DistilBertForMaskedLM.from_pretrained('distilgpt2')

# Model GPT-2 small (12 layer, 117M params)

In [3]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
model     = transformers.GPT2Model.from_pretrained('gpt2')
modelLM   = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

# Model GPT-2 medium (24 layer, 345M params)

In [ ]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2-medium')
model     = transformers.GPT2Model.from_pretrained('gpt2-medium')
modelLM   = transformers.GPT2LMHeadModel.from_pretrained('gpt2-medium')

# Model GPT-2 large (36 layer, 774M params)

In [ ]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2-large')
model     = transformers.GPT2Model.from_pretrained('gpt2-large')
modelLM   = transformers.GPT2LMHeadModel.from_pretrained('gpt2-large')

# Model CTRL huge (48 layer, 1630M params)

In [ ]:
tokenizer = transformers.CTRLTokenizer.from_pretrained('ctrl')
modelLM   = transformers.CTRLLMHeadModel.from_pretrained('ctrl')

# Language model

In [29]:
# Encode a text inputs
text = "I have two dogs and one is a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

modelLM.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
modelLM.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = modelLM(tokens_tensor)
    predictions = outputs[0]
    
# get the predicted next sub-word (in our case, the word 'man')
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
predicted_text

'I have two dogs and one is a puppy'

# Naive text generation
Only a Language Model

In [5]:
# Encode a text inputs
text   = "I have two dogs and one is a"
tokens = tokenizer.encode(text)

# Parameters
lenght = 20

for _ in range(lenght):
    tokens_tensor = torch.tensor([tokens])
    predictions   = modelLM(tokens_tensor)[0]
    next_token    = torch.argmax(predictions[0, -1, :]).item()
    tokens        += [next_token]
    
print(tokenizer.decode(tokens))
    

I have two dogs and one is a puppy. I have a dog who is a little bit older than me and I have a dog who


# Text generation
LM with repetition_penalty & temperature

- Temperature is the level of randomization. Lower value will have a high chance to output data from WebText’s test set.

In [8]:
# Encode a text inputs
text = "I have two dogs and one is a"
sequence = tokenizer.encode(text)

# Parameters
lenght = 20
repetition_penalty = 1.5
temperature = 1 #0.2

for _ in range(lenght):
    tokens_tensor = torch.tensor([sequence])
    logits = modelLM(tokens_tensor)[0]
    
    next_token_logits = logits[0, -1, :] / temperature
    
    for token in set(sequence):
        next_token_logits[token] /= repetition_penalty
        
    next_token = torch.argmax(next_token_logits).item()
    sequence += [next_token]
    
print(tokenizer.decode(sequence))
    

I have two dogs and one is a two and a two and a two and a two and a two and a two and a two and
